In [1]:
from dask.distributed import Client
from dask import compute, delayed
import dask.threaded
import dask.multiprocessing
dsch = {'d': 'distributed', 't': 'threads', 'p': 'processes', 's': 'synchronous'}
client = Client()
client

Client Scheduler: tcp://127.0.0.1:43505 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 8 Memory: 16.78 GB


In [2]:
from pprint import pprint
from pathlib import Path
import pandas as pd
import datetime as dt

In [2]:
import concurrent.futures
px = concurrent.futures.ProcessPoolExecutor
tx = concurrent.futures.ThreadPoolExecutor

In [5]:
from examinator import *

In [6]:
[*map(proc_file, filter(Path.is_file, Path('.').iterdir()))]

[{'cli.py': '03af44deec3a1f7bd1e91dff08a35060'},
 {'examinator.py': '52c251102e18e17d2e28049a69ad45b9'},
 {'scratch.ipynb': 'd20b1148de3497400b48c93f0590009a'},
 {'script.py': '3f2554bb47311b1f34231a7c0f7fe04e'},
 {'test_joblib.py': '86c0079dafa9516d998295f5f01a20c7'},
 {'__init__.py': 'ed6bac9069fd2d420ea27feb57e790e6'}]

In [4]:
values = [delayed(proc_file)(x) for x in filter(Path.is_file, Path('.').iterdir())]
values

[Delayed('proc_file-4df82a6b-b445-4a7e-a4fd-2afa6ab12222'),
 Delayed('proc_file-f1ac9465-0a95-4067-831a-fabcc6449da4'),
 Delayed('proc_file-79b028b1-e2fb-459b-8b34-1eedca862ea7'),
 Delayed('proc_file-08d42f1a-9a3a-4145-a03e-ca1d0a1854c5'),
 Delayed('proc_file-69264fd3-99df-4579-bfd9-3a5b590c8497'),
 Delayed('proc_file-27c49cde-d2a3-4224-9ac6-7ec1773f1225')]

In [10]:
func = delayed(proc_file)
vals = [*map(func, filter(Path.is_file, Path('.').iterdir()))]

results = compute(*vals, scheduler='threads')
pp(results)
%timeit compute(*vals, scheduler='threads')

len: 6

({'cli.py': '03af44deec3a1f7bd1e91dff08a35060'},
 {'examinator.py': '52c251102e18e17d2e28049a69ad45b9'},
 {'scratch.ipynb': '20af411b001c9ed8b4f6fa0744ae4bce'},
 {'script.py': '3f2554bb47311b1f34231a7c0f7fe04e'},
 {'test_joblib.py': '86c0079dafa9516d998295f5f01a20c7'},
 {'__init__.py': 'ed6bac9069fd2d420ea27feb57e790e6'})
9.02 ms ± 877 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [32]:
%timeit [*map(proc_file, filter(Path.is_file, Path('.').iterdir()))]

17 ms ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
def proc_ex(executor):
    with executor() as pool:
        results = pool.map(proc_file, filter(Path.is_file, Path('.').iterdir()))
    return results

In [39]:
%timeit [*proc_ex(concurrent.futures.ProcessPoolExecutor)]

182 ms ± 33.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [40]:
%timeit [*proc_ex(concurrent.futures.ThreadPoolExecutor)]

11.6 ms ± 1.44 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
results = compute(*values, scheduler='distributed')
pp(results)
%timeit compute(*values, scheduler='distributed')

(<map object at 0x7fda4155f2e8>,)
54 µs ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [22]:
results = compute(*values, scheduler='processes')
pp(results)
%timeit compute(*values, scheduler='processes')

len: 6

({'cli.py': '03af44deec3a1f7bd1e91dff08a35060'},
 {'examinator.py': '52c251102e18e17d2e28049a69ad45b9'},
 {'scratch.ipynb': 'ede80c6d68d8a21b817a3758b0e9165c'},
 {'script.py': '3f2554bb47311b1f34231a7c0f7fe04e'},
 {'test_joblib.py': '86c0079dafa9516d998295f5f01a20c7'},
 {'__init__.py': 'ed6bac9069fd2d420ea27feb57e790e6'})
167 ms ± 28.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
results = compute(*values, scheduler='threads')
pp(results)
%timeit compute(*values, scheduler='threads')

len: 6

({'cli.py': '03af44deec3a1f7bd1e91dff08a35060'},
 {'examinator.py': '52c251102e18e17d2e28049a69ad45b9'},
 {'scratch.ipynb': '20af411b001c9ed8b4f6fa0744ae4bce'},
 {'script.py': '3f2554bb47311b1f34231a7c0f7fe04e'},
 {'test_joblib.py': '86c0079dafa9516d998295f5f01a20c7'},
 {'__init__.py': 'ed6bac9069fd2d420ea27feb57e790e6'})
9.15 ms ± 1.18 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
def run_func(scheduler):
    it = filter(Path.is_file, Path('../..').rglob('*'))
    func = delayed(proc_file)
    vals = map(func, it)
    results = compute(*vals, scheduler=scheduler)
    return results
for s in dsch.values():
    print(s)
    %time run_func(s)

distributed


KeyboardInterrupt: 

threads


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



processes


KeyboardInterrupt: 

synchronous


In [39]:
pd.DataFrame([*map(Path.lstat, Path('.').rglob('*'))])

,0,1,2,3,4,5,6,7,8,9
0,16895,207283431,150,2,0,0,0,1551655461,1551655461,1551655461
1,33261,208755495,150,1,0,0,1846,1551656317,1551656317,1551656317
2,16895,208755576,150,2,0,0,0,1551710765,1551710765,1551710765
3,33261,208755497,150,1,0,0,4469,1551656541,1551656541,1551656541
4,16895,208755498,150,2,0,0,0,1551468724,1551468724,1551468724
5,33261,208755499,150,1,0,0,18309,1551712911,1551712911,1551712911
6,33261,208755500,150,1,0,0,681,1551130733,1551130733,1551130733
7,33261,208755501,150,1,0,0,1352,1551490946,1551490946,1551490946
8,33261,208755502,150,1,0,0,159,1550862111,1550862111,1550939939
9,16895,208755583,150,2,0,0,0,1551656573,1551656573,1551656573


In [49]:
[*map(Path.lstat, Path('./__pycache__/').rglob('*'))]

[os.stat_result(st_mode=33261, st_ino=220833437, st_dev=150, st_nlink=1, st_uid=0, st_gid=0, st_size=4140, st_atime=1551559812, st_mtime=1551559812, st_ctime=1551559812),
 os.stat_result(st_mode=33261, st_ino=220716344, st_dev=150, st_nlink=1, st_uid=0, st_gid=0, st_size=4341, st_atime=1551656572, st_mtime=1551656572, st_ctime=1551656573),
 os.stat_result(st_mode=33261, st_ino=220833439, st_dev=150, st_nlink=1, st_uid=0, st_gid=0, st_size=4095, st_atime=1551476662, st_mtime=1551476662, st_ctime=1551476662)]

In [64]:
info = Path('./__pycache__').lstat()
pp(info)

len: 10

os.stat_result(st_mode=16895, st_ino=208755583, st_dev=150, st_nlink=2, st_uid=0, st_gid=0, st_size=4096, st_atime=1551656573, st_mtime=1551656573, st_ctime=1551656573)


In [56]:
import inspect
pd.DataFrame([t for t in inspect.getmembers(Path('./__pycache__').lstat()) if not t[0].startswith('_') and not inspect.isbuiltin(t[1])], columns=['fields', 'values']).set_index('fields').T

fields,n_fields,n_sequence_fields,n_unnamed_fields,st_atime,st_atime_ns,st_blksize,st_blocks,st_ctime,st_ctime_ns,st_dev,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid
values,19.0,10.0,3.0,1.551657e+09,1.551657e+18,16384.0,8.0,1.551657e+09,1.551657e+18,150.0,0.0,208755583.0,16895.0,1.551657e+09,1.551657e+18,2.0,0.0,4096.0,0.0


In [74]:
d = dict([t for t in inspect.getmembers(info) if not t[0].startswith('_') and not inspect.isbuiltin(t[1])])
d

{'n_fields': 19,
 'n_sequence_fields': 10,
 'n_unnamed_fields': 3,
 'st_atime': 1551656573.0054905,
 'st_atime_ns': 1551656573005490600,
 'st_blksize': 16384,
 'st_blocks': 8,
 'st_ctime': 1551656573.0054905,
 'st_ctime_ns': 1551656573005490600,
 'st_dev': 150,
 'st_gid': 0,
 'st_ino': 208755583,
 'st_mode': 16895,
 'st_mtime': 1551656573.0054905,
 'st_mtime_ns': 1551656573005490600,
 'st_nlink': 2,
 'st_rdev': 0,
 'st_size': 4096,
 'st_uid': 0}

In [80]:
pd.DataFrame([d])

,n_fields,n_sequence_fields,n_unnamed_fields,st_atime,st_atime_ns,st_blksize,st_blocks,st_ctime,st_ctime_ns,st_dev,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid
0,19,10,3,1.551657e+09,1551656573005490600,16384,8,1.551657e+09,1551656573005490600,150,0,208755583,16895,1.551657e+09,1551656573005490600,2,0,4096,0


In [82]:
pd.DataFrame([*map(Path.lstat, Path('./__pycache__/').rglob('*'))])

,0,1,2,3,4,5,6,7,8,9
0,33261,220833437,150,1,0,0,4140,1551559812,1551559812,1551559812
1,33261,220716344,150,1,0,0,4341,1551656572,1551656572,1551656573
2,33261,220833439,150,1,0,0,4095,1551476662,1551476662,1551476662


In [98]:
import dask.dataframe as dd

In [100]:
dd.DataFrame?

Init signature: dd.DataFrame(dsk, name, meta, divisions)
Docstring:     
Parallel Pandas DataFrame

Do not use this class directly.  Instead use functions like
``dd.read_csv``, ``dd.read_parquet``, or ``dd.from_pandas``.

Parameters
----------
dsk: dict
    The dask graph to compute this DataFrame
name: str
    The key prefix that specifies which keys in the dask comprise this
    particular DataFrame
meta: pandas.DataFrame
    An empty ``pandas.DataFrame`` with names, dtypes, and index matching
    the expected output.
divisions: tuple of index values
    Values along which we partition our blocks on the index
File:           /opt/conda/lib/python3.7/site-packages/dask/dataframe/core.py
Type:           type
Subclasses:     


In [ ]:
%%time
def run_func(scheduler):
    it = filter(Path.is_file, Path('../../containers/').rglob('*'))
    func = delayed(proc_file)
    vals = map(func, it)
    results = compute(*vals, scheduler=scheduler)
    return results
%time run_func('processes')
%time run_func('distributed')
%time run_func('threads')

In [194]:
from functools import partial
def test(a, b):
    return a + b
test = partial(test, 1)
test(2)

3

In [196]:
from toolz import curry
@curry
def test2(a, b):
    return a + b
test1 = test2(1)
test1(3)

4

In [197]:
test2(3, 4)

7

In [178]:
def get_stat(path):
    info = path.lstat()
    d = dict([t for t in inspect.getmembers(info) if not t[0].startswith('_') and not inspect.isbuiltin(t[1])])
    d['path'] = path
    d['f_atime'] = dt.datetime.fromtimestamp(d['st_atime'])
    d['f_ctime'] = dt.datetime.fromtimestamp(d['st_ctime'])
    d['f_mtime'] = dt.datetime.fromtimestamp(d['st_mtime'])
    d['md5'] = proc_item(path)
    return d
def path_stat(path):
    return pd.DataFrame([*map(get_stat, Path(path).rglob('*'))])
path_stat('.')

In [191]:
def path_stat(path):
    return pd.DataFrame(
        compute(*map(delayed(get_stat),
                Path(path).rglob('*')),
            scheduler='threads'))
%time path_stat('..')

CPU times: user 870 ms, sys: 940 ms, total: 1.81 s
Wall time: 1.79 s


,f_atime,f_ctime,f_mtime,md5,n_fields,n_sequence_fields,n_unnamed_fields,path,st_atime,st_atime_ns,...,st_dev,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid
0,2019-02-22 18:18:20.346265,2019-02-23 16:37:47.666094,2019-02-22 18:18:20.346265,0e33fd1a5936ee17c29737b3be18b872,19,10,3,../.bak.gitignore,1.550860e+09,1550859500346264500,...,150,0,220970918,33261,1.550860e+09,1550859500346264500,1,0,17,0
1,2019-02-22 19:01:50.341002,2019-02-23 16:37:48.000095,2019-02-22 19:01:50.341002,1849f63ec7e8d6b84a5e7c2661234beb,19,10,3,../.editorconfig,1.550862e+09,1550862110341002000,...,150,0,220970919,33261,1.550862e+09,1550862110341002000,1,0,313,0
2,2019-03-02 20:41:14.451678,2019-03-02 20:41:14.451678,2019-03-02 20:41:14.451678,None,19,10,3,../.git,1.551559e+09,1551559274451678100,...,150,0,220970920,16895,1.551559e+09,1551559274451678100,2,0,0,0
3,2019-02-22 19:01:51.527860,2019-02-22 19:01:51.527860,2019-02-22 19:01:51.527860,None,19,10,3,../.github,1.550862e+09,1550862111527859900,...,150,0,220970921,16895,1.550862e+09,1550862111527859900,2,0,0,0
4,2019-02-22 19:12:29.403333,2019-02-23 16:37:47.899095,2019-02-22 19:12:29.403333,33ae75b8eb7953f5e75298b1b352deb2,19,10,3,../.gitignore,1.550863e+09,1550862749403332900,...,150,0,220970922,33261,1.550863e+09,1550862749403332900,1,0,1237,0
5,2019-02-26 17:59:44.039556,2019-02-26 17:59:44.039556,2019-02-26 17:59:44.039556,None,19,10,3,../.ipynb_checkpoints,1.551204e+09,1551203984039556000,...,150,0,220970923,16895,1.551204e+09,1551203984039556000,2,0,0,0
6,2019-02-22 19:13:56.056948,2019-02-22 19:13:55.955946,2019-02-22 19:13:55.955946,None,19,10,3,../.mypy_cache,1.550863e+09,1550862836056948200,...,150,0,220970924,16895,1.550863e+09,1550862835955946200,2,0,0,0
7,2019-02-22 19:01:50.399000,2019-02-23 16:37:54.682983,2019-02-22 19:01:50.399000,b201a82afa9a531beea191210a0c4a50,19,10,3,../.travis.yml,1.550862e+09,1550862110399000000,...,150,0,220970925,33261,1.550862e+09,1550862110399000000,1,0,314,0
8,2019-02-22 19:03:03.287219,2019-02-22 19:03:03.287219,2019-02-22 19:03:03.287219,None,19,10,3,../.vs,1.550862e+09,1550862183287219400,...,150,0,220970926,16895,1.550862e+09,1550862183287219400,2,0,4096,0
9,2019-02-24 03:17:32.610873,2019-02-24 03:17:32.610873,2019-02-24 03:17:32.610873,None,19,10,3,../.vscode,1.550978e+09,1550978252610872400,...,150,0,220970927,16895,1.550978e+09,1550978252610872400,2,0,0,0


In [166]:
ddf = dd.from_pandas(path_stat('.'), npartitions=4)

In [107]:
from toolz.functoolz import identity

In [116]:
ddf.sample(frac=1).compute()

,n_fields,n_sequence_fields,n_unnamed_fields,path,st_atime,st_atime_ns,st_blksize,st_blocks,st_ctime,st_ctime_ns,st_dev,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid
8,19,10,3,__init__.py,1.550862e+09,1550862111046862100,16384,1,1.550940e+09,1550939939162611500,150,0,208755502,33261,1.550862e+09,1550862111046862100,1,0,159,0
7,19,10,3,test_joblib.py,1.551491e+09,1551490946730578400,16384,8,1.551491e+09,1551490946730578400,150,0,208755501,33261,1.551491e+09,1551490946730578400,1,0,1352,0
4,19,10,3,scratch,1.551469e+09,1551468724500558900,16384,0,1.551469e+09,1551468724500558900,150,0,208755498,16895,1.551469e+09,1551468724500558900,2,0,0,0
10,19,10,3,.ipynb_checkpoints/cli-checkpoint.py,1.551656e+09,1551656317731657900,16384,8,1.551656e+09,1551656317847805400,150,0,208754497,33261,1.551656e+09,1551656317731657900,1,0,1846,0
2,19,10,3,dask-worker-space,1.551711e+09,1551710765589663500,16384,0,1.551711e+09,1551710765589663500,150,0,208755576,16895,1.551711e+09,1551710765589663500,2,0,0,0
3,19,10,3,examinator.py,1.551657e+09,1551656541497557700,16384,16,1.551657e+09,1551656541497557700,150,0,208755497,33261,1.551657e+09,1551656541497557700,1,0,4469,0
1,19,10,3,cli.py,1.551656e+09,1551656317731657900,16384,8,1.551656e+09,1551656317731657900,150,0,208755495,33261,1.551656e+09,1551656317731657900,1,0,1846,0
9,19,10,3,__pycache__,1.551657e+09,1551656573005490600,16384,0,1.551657e+09,1551656573005490600,150,0,208755583,16895,1.551657e+09,1551656573005490600,2,0,0,0
0,19,10,3,.ipynb_checkpoints,1.551655e+09,1551655461755122700,16384,0,1.551655e+09,1551655461755122700,150,0,207283431,16895,1.551655e+09,1551655461755122700,2,0,0,0
5,19,10,3,scratch.ipynb,1.551716e+09,1551716397328053400,16384,112,1.551716e+09,1551716397328053400,150,0,208755499,33261,1.551716e+09,1551716397328053400,1,0,54927,0


In [122]:
df

,n_fields,n_sequence_fields,n_unnamed_fields,path,st_atime,st_atime_ns,st_blksize,st_blocks,st_ctime,st_ctime_ns,st_dev,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid
0,19,10,3,__pycache__/examinator.cpython-36.pyc,1.551560e+09,1551559812751339600,16384,16,1.551560e+09,1551559812758339200,150,0,220833437,33261,1.551560e+09,1551559812751339600,1,0,4140,0
1,19,10,3,__pycache__/examinator.cpython-37.pyc,1.551657e+09,1551656572992493900,16384,16,1.551657e+09,1551656573005490600,150,0,220716344,33261,1.551657e+09,1551656572992493900,1,0,4341,0
2,19,10,3,__pycache__/examinator.cpython-38.pyc,1.551477e+09,1551476662563737300,16384,8,1.551477e+09,1551476662573737500,150,0,220833439,33261,1.551477e+09,1551476662563737300,1,0,4095,0


In [123]:
pd.to_datetime(df.st_mtime_ns)

0   2019-03-02 20:50:12.751339600
1   2019-03-03 23:42:52.992493900
2   2019-03-01 21:44:22.563737300
Name: st_mtime_ns, dtype: datetime64[ns]

In [125]:
m = df.st_mtime_ns
m

0    1551559812751339600
1    1551656572992493900
2    1551476662563737300
Name: st_mtime_ns, dtype: int64

In [132]:
df['f_atime'] = pd.to_datetime(df.st_atime_ns)
df['f_ctime'] = pd.to_datetime(df.st_ctime_ns)
df['f_mtime'] = pd.to_datetime(df.st_mtime_ns)

In [133]:
df

,n_fields,n_sequence_fields,n_unnamed_fields,path,st_atime,st_atime_ns,st_blksize,st_blocks,st_ctime,st_ctime_ns,...,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid,f_atime,f_ctime,f_mtime
0,19,10,3,__pycache__/examinator.cpython-36.pyc,1.551560e+09,1551559812751339600,16384,16,1.551560e+09,1551559812758339200,...,33261,1.551560e+09,1551559812751339600,1,0,4140,0,2019-03-02 20:50:12.751339600,2019-03-02 20:50:12.758339200,2019-03-02 20:50:12.751339600
1,19,10,3,__pycache__/examinator.cpython-37.pyc,1.551657e+09,1551656572992493900,16384,16,1.551657e+09,1551656573005490600,...,33261,1.551657e+09,1551656572992493900,1,0,4341,0,2019-03-03 23:42:52.992493900,2019-03-03 23:42:53.005490600,2019-03-03 23:42:52.992493900
2,19,10,3,__pycache__/examinator.cpython-38.pyc,1.551477e+09,1551476662563737300,16384,8,1.551477e+09,1551476662573737500,...,33261,1.551477e+09,1551476662563737300,1,0,4095,0,2019-03-01 21:44:22.563737300,2019-03-01 21:44:22.573737500,2019-03-01 21:44:22.563737300


In [142]:
dt.datetime.fromtimestamp

Docstring: Return POSIX timestamp as float.
Type:      method_descriptor


In [149]:
ddf['f_atime'] = ddf.st_atime.apply(dt.datetime.fromtimestamp)
ddf['f_ctime'] = ddf.st_ctime.apply(dt.datetime.fromtimestamp)
ddf['f_mtime'] = ddf.st_mtime.apply(dt.datetime.fromtimestamp)

/opt/conda/lib/python3.7/site-packages/dask/dataframe/core.py:2285: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  warnings.warn(msg)


In [173]:
def proc_file(path):
    return {str(path): get_md5(path)}
def get_md5(path):
    hasher = md5()
    for block in file_as_blockiter(path):
        hasher.update(block)
    return hasher.hexdigest()
def proc_item(path):
    if path.is_file():
        return get_md5(path)
    else:
        return

In [177]:
ddf = dd.from_pandas(path_stat('.'), npartitions=4)
ddf['md5'] = ddf['path'].apply(proc_item, meta=('str'))
ddf.compute()

,f_atime,f_ctime,f_mtime,n_fields,n_sequence_fields,n_unnamed_fields,path,st_atime,st_atime_ns,st_blksize,...,st_gid,st_ino,st_mode,st_mtime,st_mtime_ns,st_nlink,st_rdev,st_size,st_uid,md5
0,2019-03-03 23:24:21.755123,2019-03-03 23:24:21.755123,2019-03-03 23:24:21.755123,19,10,3,.ipynb_checkpoints,1.551655e+09,1551655461755122700,16384,...,0,207283431,16895,1.551655e+09,1551655461755122700,2,0,0,0,None
1,2019-03-03 23:38:37.731658,2019-03-03 23:38:37.731658,2019-03-03 23:38:37.731658,19,10,3,cli.py,1.551656e+09,1551656317731657900,16384,...,0,208755495,33261,1.551656e+09,1551656317731657900,1,0,1846,0,03af44deec3a1f7bd1e91dff08a35060
2,2019-03-04 14:46:05.589664,2019-03-04 14:46:05.589664,2019-03-04 14:46:05.589664,19,10,3,dask-worker-space,1.551711e+09,1551710765589663500,16384,...,0,208755576,16895,1.551711e+09,1551710765589663500,2,0,0,0,None
3,2019-03-03 23:42:21.497558,2019-03-03 23:42:21.497558,2019-03-03 23:42:21.497558,19,10,3,examinator.py,1.551657e+09,1551656541497557700,16384,...,0,208755497,33261,1.551657e+09,1551656541497557700,1,0,4469,0,52c251102e18e17d2e28049a69ad45b9
4,2019-03-01 19:32:04.500559,2019-03-01 19:32:04.500559,2019-03-01 19:32:04.500559,19,10,3,scratch,1.551469e+09,1551468724500558900,16384,...,0,208755498,16895,1.551469e+09,1551468724500558900,2,0,0,0,None
5,2019-03-04 17:08:02.829492,2019-03-04 17:08:02.829492,2019-03-04 17:08:02.829492,19,10,3,scratch.ipynb,1.551719e+09,1551719282829491700,16384,...,0,208755499,33261,1.551719e+09,1551719282829491700,1,0,234992,0,8c93a6edf7d32dbc6ce2f6d6ddad24af
6,2019-02-25 21:38:53.820684,2019-02-25 21:38:53.820684,2019-02-25 21:38:53.820684,19,10,3,script.py,1.551131e+09,1551130733820684200,16384,...,0,208755500,33261,1.551131e+09,1551130733820684200,1,0,681,0,3f2554bb47311b1f34231a7c0f7fe04e
7,2019-03-02 01:42:26.730578,2019-03-02 01:42:26.730578,2019-03-02 01:42:26.730578,19,10,3,test_joblib.py,1.551491e+09,1551490946730578400,16384,...,0,208755501,33261,1.551491e+09,1551490946730578400,1,0,1352,0,86c0079dafa9516d998295f5f01a20c7
8,2019-02-22 19:01:51.046862,2019-02-23 16:38:59.162611,2019-02-22 19:01:51.046862,19,10,3,__init__.py,1.550862e+09,1550862111046862100,16384,...,0,208755502,33261,1.550862e+09,1550862111046862100,1,0,159,0,ed6bac9069fd2d420ea27feb57e790e6
9,2019-03-03 23:42:53.005491,2019-03-03 23:42:53.005491,2019-03-03 23:42:53.005491,19,10,3,__pycache__,1.551657e+09,1551656573005490600,16384,...,0,208755583,16895,1.551657e+09,1551656573005490600,2,0,0,0,None


In [208]:
def count_dir(path):
    dirs = [*path.iterdir()]
    return len(dirs) + sum(map(count_dir, filter(Path.is_dir, dirs)))
%time count_dir(Path('../..'))
%time len([*Path('../..').rglob('*')])

CPU times: user 4.09 s, sys: 3.15 s, total: 7.24 s
Wall time: 17.6 s
CPU times: user 9.05 s, sys: 4.99 s, total: 14 s
Wall time: 48.9 s


54230

In [4]:
from toolz import curry
@curry
def pool_count(pool, path):
    dirs = [*path.iterdir()]
    return len(dirs) + sum(pool.map(pool_count(pool), filter(Path.is_dir, dirs)))
def m_count(executor, path):
    with executor() as pool:
        return pool_count(pool, Path(path))

In [23]:
from hashlib import md5
def md5_blocks(path, blocksize=1024*2048) -> str:
    path = Path(path)
    if not path.is_file():
        return
    else:
        hasher = md5()
        with path.open('rb') as file:
            block = file.read(blocksize)
            while len(block) > 0:
                hasher.update(block)
                block = file.read(blocksize)
        return hasher.hexdigest()

In [24]:
md5_blocks('cli.py')

'03af44deec3a1f7bd1e91dff08a35060'

In [9]:
def get_md5(path):
    hasher = md5()
    for block in file_as_blockiter(path):
        hasher.update(block)
    return hasher.hexdigest()

def file_as_blockiter(path, blocksize=65536):
    with open(path, 'rb') as file:
        block = file.read(blocksize)
        while len(block) > 0:
            yield block
            block = file.read(blocksize)

In [10]:
get_md5('cli.py')

'03af44deec3a1f7bd1e91dff08a35060'

In [17]:
%timeit [*map(md5_blocks, filter(Path.is_file, Path('..').rglob('*')))]
%timeit [*map(get_md5, filter(Path.is_file, Path('..').rglob('*')))]

920 ms ± 87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
1.01 s ± 202 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [13]:
%timeit [*map(lambda path: md5_blocks(path, 4096), filter(Path.is_file, Path('..').rglob('*')))]

942 ms ± 132 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%timeit [*map(lambda path: md5_blocks(path, 1024*1024), filter(Path.is_file, Path('..').rglob('*')))]
%timeit [*map(lambda path: md5_blocks(path, 1024*2048), filter(Path.is_file, Path('..').rglob('*')))]

1.04 s ± 215 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
974 ms ± 142 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
%timeit [*map(lambda path: md5_blocks(path, 65536), filter(Path.is_file, Path('..').rglob('*')))]

1.23 s ± 288 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
%timeit [*map(md5_blocks, filter(Path.is_file, Path('..').rglob('*')))]

923 ms ± 108 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
import inspect

def get_stat(path) -> dict:
    info = path.lstat()
    d = dict([t for t in inspect.getmembers(info) if not t[0].startswith('_') and not inspect.isbuiltin(t[1])])
    d['path'] = path
    d['f_atime'] = dt.datetime.fromtimestamp(d['st_atime'])
    d['f_ctime'] = dt.datetime.fromtimestamp(d['st_ctime'])
    d['f_mtime'] = dt.datetime.fromtimestamp(d['st_mtime'])
    if path.is_file():
        d['md5'] = md5_blocks(path)
    return d

def md5_blocks(path, blocksize=1024*2048) -> str:
    path = Path(path)
    if not path.is_file():
        return
    else:
        hasher = md5()
        with path.open('rb') as file:
            block = file.read(blocksize)
            while len(block) > 0:
                hasher.update(block)
                block = file.read(blocksize)
        return hasher.hexdigest()

In [40]:
path = Path('..')
%timeit _ = pd.DataFrame([*map(get_stat, Path(path).rglob('*'))])
%timeit _ = pd.DataFrame(compute(*map(delayed(get_stat), Path(path).rglob('*')), scheduler='threads'))

1.03 s ± 100 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
929 ms ± 111 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [41]:
def run_func(scheduler):
    it = filter(Path.is_file, Path('..').rglob('*'))
    func = delayed(get_stat)
    vals = map(func, it)
    results = compute(*vals, scheduler=scheduler)
    return results
for s in dsch.values():
    print(s)
    %timeit run_func(s)

distributed
1.64 s ± 272 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
threads
1.13 s ± 119 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
processes
1.71 s ± 273 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
synchronous
1.12 s ± 202 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
from boltons.iterutils import remap

In [44]:
remap?

Signature:
remap(
    root,
    visit=<function default_visit at 0x7f6d601578c8>,
    enter=<function default_enter at 0x7f6d728e4a60>,
    exit=<function default_exit at 0x7f6d728e4d90>,
    **kwargs,
)
Docstring:
The remap ("recursive map") function is used to traverse and
transform nested structures. Lists, tuples, sets, and dictionaries
are just a few of the data structures nested into heterogenous
tree-like structures that are so common in programming.
Unfortunately, Python's built-in ways to manipulate collections
are almost all flat. List comprehensions may be fast and succinct,
but they do not recurse, making it tedious to apply quick changes
or complex transforms to real-world data.

remap goes where list comprehensions cannot.

Here's an example of removing all Nones from some data:

>>> from pprint import pprint
>>> reviews = {'Star Trek': {'TNG': 10, 'DS9': 8.5, 'ENT': None},
...            'Babylon 5': 6, 'Dr. Who': None}
>>> pprint(remap(reviews, lambda p, k, v: v is not 

In [48]:
from boltons.iterutils import remap, default_enter, default_exit

def visit(path, key, value):
    return True
    #return str(path) + str(key), value

def enter(path, key, value):
    if Path(value).is_dir():
        return (), value.iterdir()

def exit(path, key, value):

remap(Path('.').iterdir())

TypeError: expected remappable root, not: <generator object Path.iterdir at 0x7f6d81dcf930>